In [0]:
%sql
--first part of silver transformation (CTE inside A CTAS, distance and minutes calculations)
CREATE TABLE silver_trip_data
USING DELTA
AS
WITH T1 AS (
    SELECT
        ride_id,
        rideable_type,
        member_casual,
        start_lat,
        start_lng,
        end_lat,
        end_lng,
        CAST(started_at AS TIMESTAMP) AS trip_start_ts,
        CAST(ended_at AS TIMESTAMP) AS trip_end_ts,
        COALESCE(start_station_name, 'UNKNOWN') AS start_station_name,
        COALESCE(start_station_id, 'N/A') AS start_station_id,
        COALESCE(end_station_name, 'UNKNOWN') AS end_station_name,
        COALESCE(end_station_id, 'N/A') AS end_station_id
    FROM
        divvy.default.bronze_trip_data
    WHERE 
        _rescued_data IS NULL
        AND started_at IS NOT NULL
        AND ended_at IS NOT NULL
),

Calculated AS (
    SELECT
        --Haversine Formula (The Haversine formula calculates the great-circle distance between two points on a sphere, given their latitudes and longitudes. It is commonly used in navigation and geospatial calculations.)
        --(https://en.wikipedia.org/wiki/Haversine_formula)
        *, 
        CASE
            WHEN trip_end_ts > trip_start_ts
            THEN ROUND(TIMESTAMPDIFF(SECOND, trip_start_ts, trip_end_ts) / 60.0, 2)
            ELSE NULL
        END AS Trip_Duration_Min,
        ROUND(
            6371 * 2 * ASIN(
                SQRT(
                    POW(SIN(RADIANS(end_lat - start_lat) / 2), 2) +
                    COS(RADIANS(start_lat)) * COS(RADIANS(end_lat)) *
                    POW(SIN(RADIANS(end_lng - start_lng) / 2), 2)
                )
            ),
            2
        ) AS Trip_Distance_Km,
        CAST(trip_start_ts AS DATE) AS Full_Date
    FROM
        T1
)

SELECT
    T2.ride_id,
    T2.rideable_type,
    T2.member_casual,
    T2.Full_Date,
    T2.start_station_id,
    T2.end_station_id,
    T2.Trip_Duration_Min,
    T2.Trip_Distance_Km,
    T2.start_lat,
    T2.start_lng,
    T2.end_lat,
    T2.end_lng,
    CAST(NULL AS STRING) AS start_community_area,
    CAST(NULL AS STRING) AS end_community_area
FROM
    Calculated AS T2
WHERE
    T2.Trip_Distance_Km > 0.05
    AND T2.Trip_Duration_Min IS NOT NULL;

In [0]:
%sql
ALTER TABLE silver_trip_data
ALTER COLUMN ride_id SET NOT NULL;

ALTER TABLE silver_trip_data
ADD CONSTRAINT silver_trip_data_pk PRIMARY KEY (ride_id);